In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from sklearn import preprocessing as pp

In [3]:
from sklearn import cross_validation as cv
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
from sklearn.svm import SVC
from sklearn import tree
from sklearn.cross_validation import cross_val_score

In [5]:
def get_final_output(clf, X, y, X_test, out_file):
    clf.fit(X, y)
    y_ = clf.predict(X_test)
    y_results = pd.DataFrame()
    
    y_results["PassengerId"] = X_test.index
    y_results["Survived"] = y_
    
    if(out_file):
        y_results.to_csv(out_file, encoding='utf-8', index=False)

In [6]:
def report_train_accuracy(clf, X, y):
    clf.fit(X, y)
    y_pred = clf.predict(X)
    return metrics.accuracy_score(y, y_pred)

In [7]:
def report_train_test_accuracy(clf, X, y, test_size, random_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test_size, random_state = random_state)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return metrics.accuracy_score(y_test, y_pred)

### Strip input data 

In [8]:
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)
print(train.shape, test.shape)

(891, 11) (418, 10)


In [9]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
cols_to_drop = ["Name", "Ticket", "Cabin"]

In [11]:
train.drop(cols_to_drop, axis= 1, inplace=True)
test.drop(cols_to_drop, axis= 1, inplace=True)

In [12]:
gender_mapping = {'female':0, 'male':1}
train['Sex'] = train['Sex'].map(gender_mapping)
test['Sex'] = test['Sex'].map(gender_mapping)

In [13]:
embarked_mapping = {'S':0, 'C':1, 'Q': 2}
train['Embarked'] = train['Embarked'].map(embarked_mapping)
test['Embarked'] = test['Embarked'].map(embarked_mapping)

In [14]:
test.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000,418.000000
mean,2.265550,0.636364,30.272590,0.447368,0.392344,35.627188,0.464115
std,0.841838,0.481622,14.181209,0.896760,0.981429,55.907576,0.685516
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,21.000000,0.000000,0.000000,7.895800,0.000000
50%,3.000000,1.000000,27.000000,0.000000,0.000000,14.454200,0.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.500000,1.000000
max,3.000000,1.000000,76.000000,8.000000,9.000000,512.329200,2.000000


In [15]:
train.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,889.000000
mean,0.383838,2.308642,0.647587,29.699118,0.523008,0.381594,32.204208,0.362205
std,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057,49.693429,0.636157
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000,7.910400,0.000000
50%,0.000000,3.000000,1.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000,31.000000,1.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000


#### Dealing with missing values

In [16]:
test.Age.fillna(inplace=True, value=test.Age.mean())
test.Fare.fillna(inplace=True, value=test.Fare.mean())

In [17]:
test.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,0.636364,30.272590,0.447368,0.392344,35.627188,0.464115
std,0.841838,0.481622,12.634534,0.896760,0.981429,55.840500,0.685516
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,23.000000,0.000000,0.000000,7.895800,0.000000
50%,3.000000,1.000000,30.272590,0.000000,0.000000,14.454200,0.000000
75%,3.000000,1.000000,35.750000,1.000000,0.000000,31.500000,1.000000
max,3.000000,1.000000,76.000000,8.000000,9.000000,512.329200,2.000000


In [18]:
train.dropna(inplace=True)
test.dropna(inplace=True)
print(train.shape, test.shape)

(712, 8) (418, 7)


In [19]:
y = train.Survived
train.drop("Survived", axis= 1, inplace=True)
X = train
X_final = test

In [20]:
print(X.shape, X_final.shape, y.shape)

(712, 7) (418, 7) (712,)


In [21]:
y.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

###  using NN to imporve the results 

In [22]:
import keras

Using TensorFlow backend.


In [23]:
keras.__version__

'2.0.9'

In [24]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD

In [25]:
batch_size = 16    #we have only 712 elements
num_classes = 2    #Survived or not
epochs=200

In [26]:
test_size = 0.2
random_state = 4

In [27]:
X.max().astype(np.float64)

Pclass        3.0000
Sex           1.0000
Age          80.0000
SibSp         5.0000
Parch         6.0000
Fare        512.3292
Embarked      2.0000
dtype: float64

In [28]:
X_max = X.append(X_final).max().astype(np.float64)
print(X_max)
X_scaled = X/X.max().astype(np.float64)
X_final  = X_final/X.max().astype(np.float64)

Pclass        3.0000
Sex           1.0000
Age          80.0000
SibSp         8.0000
Parch         9.0000
Fare        512.3292
Embarked      2.0000
dtype: float64


In [29]:
X_scaled.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,1.000000,1.0,0.2750,0.2,0.0,0.014151,0.0
2,0.333333,0.0,0.4750,0.2,0.0,0.139136,0.5
3,1.000000,0.0,0.3250,0.0,0.0,0.015469,0.0
4,0.333333,0.0,0.4375,0.2,0.0,0.103644,0.0
5,1.000000,1.0,0.4375,0.0,0.0,0.015713,0.0


In [30]:
y.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size= test_size, random_state = random_state)

In [32]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(type(X_train), type(X_test), type(y_train), type(y_test))

(569, 7) (143, 7) (569,) (143,)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [33]:
y_train = keras.utils.to_categorical(np.array(y_train), num_classes)
y_test = keras.utils.to_categorical(np.array(y_test), num_classes)

#y_train = np.array(y_train)
#y_test = np.array(y_test)

In [34]:
X_train = (np.matrix(X_train))
X_test = (np.matrix(X_test))

In [35]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(type(X_train), type(X_test), type(y_train), type(y_test))

(569, 7) (143, 7) (569, 2) (143, 2)
<class 'numpy.matrixlib.defmatrix.matrix'> <class 'numpy.matrixlib.defmatrix.matrix'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [36]:
y_train

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.]])

###  Build the Architecture

In [37]:
#Instatiaate a model
model = Sequential()

In [38]:
#Add later to it
model.add(Dense(70, activation="relu", input_shape=(7,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(2, activation='softmax'))

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 70)                560       
_________________________________________________________________
dense_2 (Dense)              (None, 70)                4970      
_________________________________________________________________
dense_3 (Dense)              (None, 70)                4970      
_________________________________________________________________
dense_4 (Dense)              (None, 70)                4970      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 142       
Total params: 15,612
Trainable params: 15,612
Non-trainable params: 0
_________________________________________________________________


In [40]:
sgd = SGD(lr= 0.01)

In [41]:
#Complie the model for running
model.compile(loss='categorical_crossentropy',
             optimizer = sgd,
             metrics =['accuracy'])

In [42]:
model.fit(X_train, y_train, batch_size= batch_size, epochs=180, verbose=1, validation_data=(X_test, y_test))

Train on 569 samples, validate on 143 samples
Epoch 1/180
569/569 [==============================] - 0s 304us/step - loss: 0.6777 - acc: 0.5923 - val_loss: 0.6647 - val_acc: 0.6084
Epoch 2/180
569/569 [==============================] - 0s 302us/step - loss: 0.6611 - acc: 0.5923 - val_loss: 0.6495 - val_acc: 0.6084
Epoch 3/180
569/569 [==============================] - 0s 197us/step - loss: 0.6479 - acc: 0.5923 - val_loss: 0.6360 - val_acc: 0.6084
Epoch 4/180
569/569 [==============================] - 0s 195us/step - loss: 0.6348 - acc: 0.5923 - val_loss: 0.6227 - val_acc: 0.6503
Epoch 5/180
569/569 [==============================] - 0s 303us/step - loss: 0.6216 - acc: 0.6696 - val_loss: 0.6087 - val_acc: 0.7413
Epoch 6/180
569/569 [==============================] - 0s 201us/step - loss: 0.6083 - acc: 0.7522 - val_loss: 0.5949 - val_acc: 0.7343
Epoch 7/180
569/569 [==============================] - 0s 194us/step - loss: 0.5947 - acc: 0.7750 - val_loss: 0.5808 - val_acc: 0.7622
Epoch 8/1

Epoch 61/180
569/569 [==============================] - 0s 195us/step - loss: 0.4351 - acc: 0.8084 - val_loss: 0.4086 - val_acc: 0.8252
Epoch 62/180
569/569 [==============================] - 0s 199us/step - loss: 0.4377 - acc: 0.7979 - val_loss: 0.4080 - val_acc: 0.8322
Epoch 63/180
569/569 [==============================] - 0s 303us/step - loss: 0.4318 - acc: 0.8084 - val_loss: 0.4100 - val_acc: 0.8182
Epoch 64/180
569/569 [==============================] - 0s 195us/step - loss: 0.4318 - acc: 0.8084 - val_loss: 0.4156 - val_acc: 0.8252
Epoch 65/180
569/569 [==============================] - 0s 197us/step - loss: 0.4365 - acc: 0.7996 - val_loss: 0.4095 - val_acc: 0.8182
Epoch 66/180
569/569 [==============================] - 0s 306us/step - loss: 0.4329 - acc: 0.8137 - val_loss: 0.4055 - val_acc: 0.8322
Epoch 67/180
569/569 [==============================] - 0s 197us/step - loss: 0.4346 - acc: 0.8084 - val_loss: 0.4046 - val_acc: 0.8322
Epoch 68/180
569/569 [==========================

569/569 [==============================] - 0s 199us/step - loss: 0.4176 - acc: 0.8278 - val_loss: 0.3933 - val_acc: 0.8322
Epoch 122/180
569/569 [==============================] - 0s 306us/step - loss: 0.4146 - acc: 0.8243 - val_loss: 0.3887 - val_acc: 0.8182
Epoch 123/180
569/569 [==============================] - 0s 196us/step - loss: 0.4183 - acc: 0.8155 - val_loss: 0.3906 - val_acc: 0.8252
Epoch 124/180
569/569 [==============================] - 0s 195us/step - loss: 0.4219 - acc: 0.8155 - val_loss: 0.3928 - val_acc: 0.8322
Epoch 125/180
569/569 [==============================] - 0s 305us/step - loss: 0.4127 - acc: 0.8295 - val_loss: 0.3886 - val_acc: 0.8392
Epoch 126/180
569/569 [==============================] - 0s 196us/step - loss: 0.4163 - acc: 0.8243 - val_loss: 0.3909 - val_acc: 0.8322
Epoch 127/180
569/569 [==============================] - 0s 196us/step - loss: 0.4158 - acc: 0.8295 - val_loss: 0.3925 - val_acc: 0.8322
Epoch 128/180
569/569 [==============================] 

In [43]:
score = model.evaluate(X_test, y_test, verbose=0)

In [44]:
print('Test Loss', score[0])
print('Test Accuracy', score[1])

Test Loss 0.397946185791
Test Accuracy 0.839160839578


### Retrain the model 

In [45]:
X_scaled = np.matrix(X_scaled)
y_scaled = keras.utils.to_categorical(np.array(y), num_classes)
print(type(X_scaled), type(y_scaled))

<class 'numpy.matrixlib.defmatrix.matrix'> <class 'numpy.ndarray'>


In [46]:
model = Sequential()

#Add later to it
model.add(Dense(70, activation="relu", input_shape=(7,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer = sgd,
             metrics =['accuracy'])

model.fit(X_scaled, y_scaled, batch_size= batch_size, epochs=220, verbose=1)

Epoch 1/220
712/712 [==============================] - 0s 325us/step - loss: 0.7070 - acc: 0.4354
Epoch 2/220
712/712 [==============================] - 0s 166us/step - loss: 0.6682 - acc: 0.5829
Epoch 3/220
712/712 [==============================] - 0s 254us/step - loss: 0.6497 - acc: 0.5815 0s - loss: 0.6497 - acc: 0.581
Epoch 4/220
712/712 [==============================] - 0s 168us/step - loss: 0.6347 - acc: 0.5801
Epoch 5/220
712/712 [==============================] - 0s 255us/step - loss: 0.6190 - acc: 0.5969
Epoch 6/220
712/712 [==============================] - 0s 167us/step - loss: 0.6010 - acc: 0.7317
Epoch 7/220
712/712 [==============================] - 0s 254us/step - loss: 0.5833 - acc: 0.7711
Epoch 8/220
712/712 [==============================] - 0s 168us/step - loss: 0.5657 - acc: 0.7795
Epoch 9/220
712/712 [==============================] - 0s 256us/step - loss: 0.5503 - acc: 0.7795
Epoch 10/220
712/712 [==============================] - 0s 168us/step - loss: 0.5374 - 

712/712 [==============================] - 0s 167us/step - loss: 0.4121 - acc: 0.8216
Epoch 82/220
712/712 [==============================] - 0s 258us/step - loss: 0.4142 - acc: 0.8315 0s - loss: 0.4839 - acc: 0.76
Epoch 83/220
712/712 [==============================] - 0s 171us/step - loss: 0.4137 - acc: 0.8202
Epoch 84/220
712/712 [==============================] - 0s 256us/step - loss: 0.4121 - acc: 0.8287
Epoch 85/220
712/712 [==============================] - 0s 257us/step - loss: 0.4076 - acc: 0.8287
Epoch 86/220
712/712 [==============================] - 0s 167us/step - loss: 0.4098 - acc: 0.8315
Epoch 87/220
712/712 [==============================] - 0s 256us/step - loss: 0.4120 - acc: 0.8329
Epoch 88/220
712/712 [==============================] - 0s 166us/step - loss: 0.4103 - acc: 0.8329
Epoch 89/220
712/712 [==============================] - 0s 256us/step - loss: 0.4079 - acc: 0.8385 0s - loss: 0.4163 - acc: 0.828
Epoch 90/220
712/712 [==============================] - 0s 16

712/712 [==============================] - 0s 168us/step - loss: 0.3927 - acc: 0.8385 0s - loss: 0.4257 - acc: 0.815
Epoch 161/220
712/712 [==============================] - 0s 258us/step - loss: 0.3915 - acc: 0.8343
Epoch 162/220
712/712 [==============================] - 0s 169us/step - loss: 0.3946 - acc: 0.8357
Epoch 163/220
712/712 [==============================] - 0s 255us/step - loss: 0.3941 - acc: 0.8385
Epoch 164/220
712/712 [==============================] - 0s 170us/step - loss: 0.3957 - acc: 0.8371
Epoch 165/220
712/712 [==============================] - 0s 250us/step - loss: 0.3934 - acc: 0.8357
Epoch 166/220
712/712 [==============================] - 0s 253us/step - loss: 0.3958 - acc: 0.8357
Epoch 167/220
712/712 [==============================] - 0s 166us/step - loss: 0.3959 - acc: 0.8343
Epoch 168/220
712/712 [==============================] - 0s 258us/step - loss: 0.3941 - acc: 0.8343
Epoch 169/220
712/712 [==============================] - 0s 168us/step - loss: 0.39

In [47]:
X_final = np.matrix(X_final)

print(X_final.shape, type(X_final), )

(418, 7) <class 'numpy.matrixlib.defmatrix.matrix'>


In [48]:
y_final_pred = model.predict_classes(X_final)

418/418 [==============================] - 0s 82us/step


In [49]:
y_final_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [50]:
y_final = pd.DataFrame()

In [51]:
print(y_final_pred.shape, y_final.shape)

(418,) (0, 0)


In [52]:
y_final.head()

""


In [53]:
y_final["PassengerId"] = test.index
y_final["Survived"] = pd.Series(y_final_pred)

In [54]:
y_final.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [55]:
y_final.to_csv("titanic_results_06_nn.csv", encoding='utf-8', index=False)

### Building a complex NN model

In [56]:
model = Sequential()

#Add later to it
model.add(Dense(700, activation="relu", input_shape=(7,)))
model.add(Dense(700, activation="relu", input_shape=(700,)))
model.add(Dense(700, activation="relu", input_shape=(700,)))
model.add(Dense(700, activation="relu", input_shape=(700,)))
model.add(Dense(70, activation="relu", input_shape=(700,)))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer = sgd,
             metrics =['accuracy'])



In [57]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 700)               5600      
_________________________________________________________________
dense_12 (Dense)             (None, 700)               490700    
_________________________________________________________________
dense_13 (Dense)             (None, 700)               490700    
_________________________________________________________________
dense_14 (Dense)             (None, 700)               490700    
_________________________________________________________________
dense_15 (Dense)             (None, 70)                49070     
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 142       
Total params: 1,526,912
Trainable params: 1,526,912
Non-trainable params: 0
_________________________________________________________________


In [58]:
model.fit(X_train, y_train, batch_size= batch_size, epochs=300, verbose=1, validation_data=(X_test, y_test))

Train on 569 samples, validate on 143 samples
Epoch 1/300
569/569 [==============================] - 2s 3ms/step - loss: 0.6736 - acc: 0.6081 - val_loss: 0.6555 - val_acc: 0.6084
Epoch 2/300
569/569 [==============================] - 1s 3ms/step - loss: 0.6490 - acc: 0.5923 - val_loss: 0.6340 - val_acc: 0.6084
Epoch 3/300
569/569 [==============================] - 1s 3ms/step - loss: 0.6301 - acc: 0.5923 - val_loss: 0.6153 - val_acc: 0.6084
Epoch 4/300
569/569 [==============================] - 1s 3ms/step - loss: 0.6123 - acc: 0.6643 - val_loss: 0.5966 - val_acc: 0.6853
Epoch 5/300
569/569 [==============================] - 1s 3ms/step - loss: 0.5937 - acc: 0.7522 - val_loss: 0.5774 - val_acc: 0.7762s - loss: 0.5941 - acc: 0
Epoch 6/300
569/569 [==============================] - 2s 3ms/step - loss: 0.5757 - acc: 0.7768 - val_loss: 0.5588 - val_acc: 0.7902
Epoch 7/300
569/569 [==============================] - 2s 3ms/step - loss: 0.5578 - acc: 0.7856 - val_loss: 0.5426 - val_acc: 0.783

569/569 [==============================] - 2s 3ms/step - loss: 0.4271 - acc: 0.8155 - val_loss: 0.4007 - val_acc: 0.8462
Epoch 62/300
569/569 [==============================] - 2s 3ms/step - loss: 0.4245 - acc: 0.8155 - val_loss: 0.4063 - val_acc: 0.8252
Epoch 63/300
569/569 [==============================] - 2s 3ms/step - loss: 0.4220 - acc: 0.8295 - val_loss: 0.4314 - val_acc: 0.8252
Epoch 64/300
569/569 [==============================] - 2s 3ms/step - loss: 0.4311 - acc: 0.8207 - val_loss: 0.4008 - val_acc: 0.8462
Epoch 65/300
569/569 [==============================] - 2s 3ms/step - loss: 0.4253 - acc: 0.8155 - val_loss: 0.3989 - val_acc: 0.8392
Epoch 66/300
569/569 [==============================] - 2s 3ms/step - loss: 0.4256 - acc: 0.8155 - val_loss: 0.4002 - val_acc: 0.8112
Epoch 67/300
569/569 [==============================] - 2s 3ms/step - loss: 0.4229 - acc: 0.8225 - val_loss: 0.3982 - val_acc: 0.8182
Epoch 68/300
569/569 [==============================] - 2s 3ms/step - loss:

569/569 [==============================] - 2s 3ms/step - loss: 0.4050 - acc: 0.8330 - val_loss: 0.4021 - val_acc: 0.8462
Epoch 122/300
569/569 [==============================] - 2s 3ms/step - loss: 0.4003 - acc: 0.8278 - val_loss: 0.3941 - val_acc: 0.8322
Epoch 123/300
569/569 [==============================] - 2s 3ms/step - loss: 0.4016 - acc: 0.8243 - val_loss: 0.3931 - val_acc: 0.8322
Epoch 124/300
569/569 [==============================] - 2s 3ms/step - loss: 0.4039 - acc: 0.8366 - val_loss: 0.3979 - val_acc: 0.8252
Epoch 125/300
569/569 [==============================] - 2s 3ms/step - loss: 0.4046 - acc: 0.8278 - val_loss: 0.4129 - val_acc: 0.8042
Epoch 126/300
569/569 [==============================] - 2s 3ms/step - loss: 0.4016 - acc: 0.8313 - val_loss: 0.4008 - val_acc: 0.8252
Epoch 127/300
569/569 [==============================] - 2s 3ms/step - loss: 0.3957 - acc: 0.8313 - val_loss: 0.4034 - val_acc: 0.8112
Epoch 128/300
569/569 [==============================] - 2s 3ms/step 

569/569 [==============================] - 1s 3ms/step - loss: 0.3868 - acc: 0.8348 - val_loss: 0.4190 - val_acc: 0.7832TA: 0s - loss: 0.3548 - acc:
Epoch 182/300
569/569 [==============================] - 1s 3ms/step - loss: 0.3888 - acc: 0.8366 - val_loss: 0.4381 - val_acc: 0.7762
Epoch 183/300
569/569 [==============================] - 1s 3ms/step - loss: 0.3862 - acc: 0.8348 - val_loss: 0.4143 - val_acc: 0.8042
Epoch 184/300
569/569 [==============================] - 2s 3ms/step - loss: 0.3837 - acc: 0.8330 - val_loss: 0.4095 - val_acc: 0.8322
Epoch 185/300
569/569 [==============================] - 1s 3ms/step - loss: 0.3840 - acc: 0.8348 - val_loss: 0.4129 - val_acc: 0.8322
Epoch 186/300
569/569 [==============================] - 1s 3ms/step - loss: 0.3885 - acc: 0.8401 - val_loss: 0.4157 - val_acc: 0.8042
Epoch 187/300
569/569 [==============================] - 1s 3ms/step - loss: 0.3851 - acc: 0.8401 - val_loss: 0.4064 - val_acc: 0.8322
Epoch 188/300
569/569 [==================

Epoch 240/300
569/569 [==============================] - 1s 3ms/step - loss: 0.3627 - acc: 0.8436 - val_loss: 0.4399 - val_acc: 0.7832
Epoch 241/300
569/569 [==============================] - 1s 3ms/step - loss: 0.3740 - acc: 0.8436 - val_loss: 0.4291 - val_acc: 0.8182
Epoch 242/300
569/569 [==============================] - 2s 3ms/step - loss: 0.3729 - acc: 0.8383 - val_loss: 0.4455 - val_acc: 0.7622 - loss: 0.3590 - acc:
Epoch 243/300
569/569 [==============================] - 2s 3ms/step - loss: 0.3668 - acc: 0.8594 - val_loss: 0.4476 - val_acc: 0.8322
Epoch 244/300
569/569 [==============================] - 1s 3ms/step - loss: 0.3786 - acc: 0.8313 - val_loss: 0.4368 - val_acc: 0.7692
Epoch 245/300
569/569 [==============================] - 1s 3ms/step - loss: 0.3712 - acc: 0.8418 - val_loss: 0.4426 - val_acc: 0.7413
Epoch 246/300
569/569 [==============================] - 1s 2ms/step - loss: 0.3757 - acc: 0.8541 - val_loss: 0.4333 - val_acc: 0.7972
Epoch 247/300
569/569 [==========

Epoch 300/300
569/569 [==============================] - 1s 3ms/step - loss: 0.3594 - acc: 0.8366 - val_loss: 0.4373 - val_acc: 0.7972


In [59]:
model = Sequential()

#Add later to it
model.add(Dense(700, activation="relu", input_shape=(7,)))
model.add(Dense(700, activation="relu", input_shape=(700,)))
model.add(Dense(700, activation="relu", input_shape=(700,)))
model.add(Dense(700, activation="relu", input_shape=(700,)))
model.add(Dense(70, activation="relu", input_shape=(700,)))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer = sgd,
             metrics =['accuracy'])

In [60]:
model.fit(X_scaled, y_scaled, batch_size= batch_size, epochs=220, verbose=1)

Epoch 1/220
712/712 [==============================] - 2s 3ms/step - loss: 0.6624 - acc: 0.5955A: 1s - loss: 0.6809 - a
Epoch 2/220
712/712 [==============================] - 2s 3ms/step - loss: 0.6357 - acc: 0.5955
Epoch 3/220
712/712 [==============================] - 2s 3ms/step - loss: 0.6136 - acc: 0.5955A: 1s - loss: 0.6251 - 
Epoch 4/220
712/712 [==============================] - 2s 2ms/step - loss: 0.5913 - acc: 0.7093A: 0s - loss: 0.6007 - acc
Epoch 5/220
712/712 [==============================] - 2s 3ms/step - loss: 0.5689 - acc: 0.7739A: 1s - loss: 0.5802 -
Epoch 6/220
712/712 [==============================] - 2s 3ms/step - loss: 0.5495 - acc: 0.7851A: 1s - loss: 0.5528 -  - ETA: 0s - loss: 0.5503 - acc: 0.784
Epoch 7/220
712/712 [==============================] - 2s 3ms/step - loss: 0.5350 - acc: 0.7809A: 0s - loss: 0.5472 - acc: 
Epoch 8/220
712/712 [==============================] - 2s 3ms/step - loss: 0.5229 - acc: 0.7809
Epoch 9/220
712/712 [===========================

712/712 [==============================] - 2s 3ms/step - loss: 0.4134 - acc: 0.8272
Epoch 72/220
712/712 [==============================] - 2s 3ms/step - loss: 0.4135 - acc: 0.8244A: 1s - loss: 0.4
Epoch 73/220
712/712 [==============================] - 2s 3ms/step - loss: 0.4103 - acc: 0.8272
Epoch 74/220
712/712 [==============================] - 2s 3ms/step - loss: 0.4108 - acc: 0.8272
Epoch 75/220
712/712 [==============================] - 2s 3ms/step - loss: 0.4123 - acc: 0.8287
Epoch 76/220
712/712 [==============================] - 2s 3ms/step - loss: 0.4079 - acc: 0.8287A: 1s - loss: 0.3833 
Epoch 77/220
712/712 [==============================] - 2s 2ms/step - loss: 0.4079 - acc: 0.8343
Epoch 78/220
712/712 [==============================] - 2s 2ms/step - loss: 0.4097 - acc: 0.8287A: 0s - loss: 0.4028 - acc:
Epoch 79/220
712/712 [==============================] - 2s 3ms/step - loss: 0.4077 - acc: 0.8329
Epoch 80/220
712/712 [==============================] - 2s 2ms/step - loss:

Epoch 145/220
712/712 [==============================] - 2s 2ms/step - loss: 0.3928 - acc: 0.8413
Epoch 146/220
712/712 [==============================] - 2s 3ms/step - loss: 0.3945 - acc: 0.8343
Epoch 147/220
712/712 [==============================] - 2s 3ms/step - loss: 0.3975 - acc: 0.8357
Epoch 148/220
712/712 [==============================] - 2s 3ms/step - loss: 0.3955 - acc: 0.8385A: 0s - loss: 0.4125 - acc: 0.832 - ETA: 0s - loss: 0.3989 - acc: 0.
Epoch 149/220
712/712 [==============================] - 2s 3ms/step - loss: 0.3915 - acc: 0.8343
Epoch 150/220
712/712 [==============================] - 2s 3ms/step - loss: 0.3922 - acc: 0.8287
Epoch 151/220
712/712 [==============================] - 2s 3ms/step - loss: 0.3966 - acc: 0.8272A: 0s - loss: 0.3956 - acc: 0.825
Epoch 152/220
712/712 [==============================] - 2s 2ms/step - loss: 0.3957 - acc: 0.8329
Epoch 153/220
712/712 [==============================] - 2s 3ms/step - loss: 0.3946 - acc: 0.8357
Epoch 154/220
712

712/712 [==============================] - 2s 3ms/step - loss: 0.3840 - acc: 0.8385A: 0s - loss: 0.3940 - acc: 0.8
Epoch 214/220
712/712 [==============================] - ETA: 0s - loss: 0.3867 - acc: 0.840 - 2s 3ms/step - loss: 0.3807 - acc: 0.8455
Epoch 215/220
712/712 [==============================] - 2s 2ms/step - loss: 0.3819 - acc: 0.8385
Epoch 216/220
712/712 [==============================] - 2s 3ms/step - loss: 0.3853 - acc: 0.8301A: 0s - loss: 0.3965 - acc: 
Epoch 217/220
712/712 [==============================] - 2s 2ms/step - loss: 0.3762 - acc: 0.8483A: 0s - loss: 0.3293 - ac
Epoch 218/220
712/712 [==============================] - 2s 3ms/step - loss: 0.3863 - acc: 0.8371
Epoch 219/220
712/712 [==============================] - 2s 3ms/step - loss: 0.3849 - acc: 0.8427A: 0s - loss: 0.3840 - acc: 0.8 - ETA: 0s - loss: 0.3817 - acc: 0.8 - ETA: 0s - loss: 0.3740 - acc: 0.850
Epoch 220/220
712/712 [==============================] - 2s 3ms/step - loss: 0.3852 - acc: 0.8287A: 0

In [61]:
y_final_pred = model.predict_classes(X_final)

418/418 [==============================] - 0s 696us/step


In [62]:
y_final = pd.DataFrame()

In [63]:
y_final["PassengerId"] = test.index
y_final["Survived"] = pd.Series(y_final_pred)

In [64]:
y_final.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [65]:
y_final.to_csv("titanic_results_07_nn_complex.csv", encoding='utf-8', index=False)